# Portfolio Optimization


## Pre-processing step: Estimate expected returns and covariance

In [69]:
# Import gurobi and numpy
from gurobipy import *
import numpy as np
from numpy import genfromtxt
import csv

## Get index of 4 tickers
tick4 = ["MSFT","GS","PG","SCHP"];

# Get variable names
with open('/Users/estherchung/Downloads/Prices.csv') as csvFile:
    reader = csv.reader(csvFile)
    tickers = next(reader) ## stores the tickers of all 390 stocks

tickind =[];
for t in tick4:
    tickind.append(tickers.index(t)) ## retrieve index that corresponds to each ticker

# Load data
prices = genfromtxt('/Users/estherchung/Downloads/Prices.csv', delimiter=',',skip_header = 1)

# get dimensions of data
d = prices.shape[0]
n = prices.shape[1]

# calculate monthly returns of each stock
returns = np.zeros((d-1,n))
for stock in range(n):
    for month in range(d-1):
        returns[month,stock] = prices[month+1,stock]/prices[month,stock]-1
        
# Store average return (parameter r_i in portfolio optimization model)       
avg_return = np.zeros(n)
avg_return = np.mean(returns,axis=0)

# Store covariance matrix (parameter C_ij in portfolio optimization model)
C = np.zeros((n,n))
C = np.cov(np.transpose(returns))

### a) Model 1

In [70]:
from gurobipy import *

m = Model("Minimum Variance Portfolio - Model 1")
x = m.addVars(n, name="x")
m.setObjective(sum(sum(C[i,j]*x[i]*x[j] for i in range(n)) for j in range(n)), GRB.MINIMIZE)
m.addConstr(sum(avg_return[i]*x[i] for i in range(n)) >= 0.005)
m.addConstr(x.sum() == 1, "budget")
for i in range(n):
    if i not in tickind:
        x[i].ub = 0
m.update()
m.optimize()
for i in range(n):
    if x[i].x > 0:
        print(tickers[i], x[i].x)
print("The Optimal Risk is: ", m.objVal)
print("Runtime: ", m.Runtime)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 390 columns and 780 nonzeros
Model fingerprint: 0x9d742184
Model has 76245 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 0 rows and 386 columns
Presolve time: 0.03s
Presolved: 2 rows, 4 columns, 8 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   4.96102155e+05 -4.96102155e+05  3.50e+03 1.9

### b) Model 2

In [71]:
m = Model("Minimum Variance Portfolio - Model 2")
x = {}
for i in range(n):
    x[i] = m.addVar(name="x_%s" % i)
m.addConstr(sum(x[i] for i in range(n)) == 1)
m.addConstr(sum(avg_return[i]*x[i] for i in range(n)) >= 0.005)
m.setObjective(sum(sum(C[i][j]*x[i]*x[j] for i in range(n)) for j in range(n)), GRB.MINIMIZE)
m.optimize()
print("The Optimal Risk is: ", m.objVal)
print("Runtime: ", m.Runtime)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 390 columns and 780 nonzeros
Model fingerprint: 0xb0353995
Model has 76245 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve time: 0.01s
Presolved: 2 rows, 390 columns, 780 nonzeros
Presolved model has 76245 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 59
 AA' NZ     : 1.830e+03
 Factor NZ  : 1.891e+03
 Factor Ops : 7.753e+04 (less than 1 second per iteration)
 Threads    : 8

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.89821559e-13 -2.89821559e-13  3.90e+05 3.22e-07  1.00e+06     0s
   1   2.

### c) Model 3

In [76]:
m = Model("Minimum Variance Portfolio - Model 3")
x = m.addVars(n, name="x")
z = m.addVars(n, vtype=GRB.BINARY, name="z")
m.setObjective(sum(x[i]*C[i,j]*x[j] for i in range(n) for j in range(n))/2, GRB.MINIMIZE)

m.addConstr(sum(x[i] for i in range(n)) == 1)
m.addConstr(sum(z[i] for i in range(n)) <= 4)
m.addConstrs((x[i] <= z[i] * GRB.INFINITY for i in range(n)))
m.addConstr(sum(avg_return[i]*x[i] for i in range(n)) >= 0.005, "expected_return")

m.optimize()
print("The Optimal Risk is: ", m.objVal)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 393 rows, 780 columns and 1950 nonzeros
Model fingerprint: 0x09de752c
Model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+100]
  Objective range  [0e+00, 0e+00]
  QObjective range [8e-08, 4e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve time: 0.02s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Found heuristic solution: objective 0.0004516

Root relaxation: objective 1.439274e-05, 103 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current N

In [77]:
prices = pd.read_csv('Downloads/Prices.csv')
ticker_lst = prices.columns
tickind =[]
for t in ticker_lst:
    tickind.append(tickers.index(t))

for v in m.getVars():
    if v.x > 1e-6:
        ticker_index = int(v.varName[2:5])
        print(f'{ticker_lst[ticker_index]} = {v.x}')
        
print("Runtime: ", m.Runtime)

CME = 0.12641141929490474
LLY = 0.07547619035437454
NVDA = 0.04375370672843181
BND = 0.7543586836222889
CME = 1.0
LLY = 1.0
NVDA = 1.0
BND = 1.0
Runtime:  114.65288209915161



* For model 1, The Optimal Risk is: 0.000177
* For model 2, The Optimal Risk is: 0.000029

The optimal risk in Model 1 is higher than Model 2. This is because, in Model 1, we only allowed the portfolio to invest in 4 stocks, which limits the diversification of the portfolio and therefore increases the risk. In Model 2, the portfolio can invest in all 390 stocks, which increases the diversification of the portfolio and therefore decreases the risk.


* For model 2, The Optimal Risk is: 0.000029
* For model 3, The Optimal Risk is: 0.000037

Model 3 has a higher optimal risk because it is constrained to hold at most 4 stocks, while Model 2 is not constrained in the number of stocks it holds. This means that Model 2 can diversify its portfolio over a larger number of stocks, reducing the portfolio's risk. The constraint in Model 3 to hold at most 4 stocks reduces the portfolio's ability to diversify and increase its risk. Additionally, by limiting the number of stocks to 4, we are losing the benefits of having a more diversified portfolio, which results in higher risk.

In [78]:
m = Model("Minimum Variance Portfolio - Model 3")
x = m.addVars(n, name="x")
z = m.addVars(n, vtype=GRB.BINARY, name="z")
m.setObjective(sum(x[i]*C[i,j]*x[j] for i in range(n) for j in range(n))/2, GRB.MINIMIZE)

m.addConstr(sum(x[i] for i in range(n)) == 1)
m.addConstr(sum(z[i] for i in range(n)) <= 4)
m.addConstrs((x[i] <= z[i] * GRB.INFINITY for i in range(n)))
m.addConstr(sum(avg_return[i]*x[i] for i in range(n)) >= 0.005, "expected_return")
m.Params.TimeLimit = 30.0
m.optimize()
print("The Optimal Risk is: ", m.objVal)
print("Runtime: ", m.Runtime)

Set parameter TimeLimit to value 30
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 393 rows, 780 columns and 1950 nonzeros
Model fingerprint: 0x09de752c
Model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+100]
  Objective range  [0e+00, 0e+00]
  QObjective range [8e-08, 4e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve time: 0.01s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Found heuristic solution: objective 0.0004516

Root relaxation: objective 1.439274e-05, 103 iterations, 0.01 seconds (0.00 work

**The value of Optimal risk is almost the same**

In [79]:
m = Model("Minimum Variance Portfolio - Model 3")
x = m.addVars(n, name="x")
z = m.addVars(n, vtype=GRB.BINARY, name="z")
m.setObjective(sum(x[i]*C[i,j]*x[j] for i in range(n) for j in range(n))/2, GRB.MINIMIZE)

m.addConstr(sum(x[i] for i in range(n)) == 1)
m.addConstr(sum(z[i] for i in range(n)) <= 4)
m.addConstrs((x[i] <= z[i] * GRB.INFINITY for i in range(n)))
m.addConstr(sum(avg_return[i]*x[i] for i in range(n)) >= 0.005, "expected_return")
m.Params.MIPGap = 0.1
m.optimize()
print("The Optimal Risk is: ", m.objVal)
print("Runtime: ", m.Runtime)

Set parameter MIPGap to value 0.1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 393 rows, 780 columns and 1950 nonzeros
Model fingerprint: 0x09de752c
Model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+100]
  Objective range  [0e+00, 0e+00]
  QObjective range [8e-08, 4e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve time: 0.01s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Found heuristic solution: objective 0.0004516

Root relaxation: objective 1.439274e-05, 103 iterations, 0.00 seconds (0.00 work u

**The runtime is approximately 20 seconds shorter than the solution time obtained in question 1c**